# Imports and technical setup

In [1]:
from pathlib import Path
persist_path = Path('..') / 'persist'
import numpy as np
import pandas as pd
from pandas import IndexSlice as idx
from IPython.display import display
import matplotlib.pyplot as plt
import sys
project_root = str(Path(sys.path[0]).parents[0].absolute())
if project_root not in sys.path:
    sys.path.append(project_root)
from importlib import reload
import scripts.utils as utils
from bokeh.io import output_notebook, show
output_notebook()
from functools import partial

Loading BokehJS ...

# Construction d'un indicateur "taux de web"

In [42]:
orders = pd.read_pickle(persist_path / 'orders.pkl')
# orders

In [2]:
day_orders = pd.read_pickle(persist_path / 'day_order.pkl')
# day_orders

indicators                    weight                    margin               \
origin2                          EDI       TV   VR  WEB    EDI      TV   VR   
orgacom client     date                                                       
1ALO    0000015304 2017-07-03    0.0  190.642  0.0  0.0    0.0  202.80  0.0   
                   2017-07-04    0.0   24.200  0.0  0.0    0.0   33.70  0.0   
                   2017-07-05    0.0  112.550  0.0  0.0    0.0   86.73  0.0   
                   2017-07-06    0.0    6.200  0.0  0.0    0.0   17.55  0.0   
                   2017-07-07    0.0   27.000  0.0  0.0    0.0   13.77  0.0   
...                              ...      ...  ...  ...    ...     ...  ...   
2BRE    SL009N     2020-08-26    0.0    0.000  0.0  0.0    0.0    0.00  0.0   
                   2020-08-27    0.0  103.950  0.0  0.0    0.0   61.10  0.0   
                   2020-08-28    0.0    0.000  0.0  0.0    0.0    0.00  0.0   
                   2020-08-31    0.0   40.500  0.0  0.0    0.0   10.51  0.0   
        SLD008B    2018-09-14    0.0    4.400  0.0  0.0    0.0    3.82  0.0   

indicators                         brutrevenue                     
origin2                        WEB         EDI       TV   VR  WEB  
orgacom client     date                                            
1ALO    0000015304 2017-07-03  0.0         0.0  1340.95  0.0  0.0  
                   2017-07-04  0.0         0.0   196.54  0.0  0.0  
                   2017-07-05  0.0         0.0   916.78  0.0  0.0  
                   2017-07-06  0.0         0.0    73.36  0.0  0.0  
                   2017-07-07  0.0         0.0    73.17  0.0  0.0  
...                            ...         ...      ...  ...  ...  
2BRE    SL009N     2020-08-26  0.0         0.0     0.00  0.0  0.0  
                   2020-08-27  0.0         0.0   629.91  0.0  0.0  
                   2020-08-28  0.0         0.0     0.00  0.0  0.0  
                   2020-08-31  0.0         0.0    75.03  0.0  0.0  
        SLD008B    2018-09-14  0.0         0.0    70.05  0.0  0.0  

[15650027 rows x 12 columns]

In [3]:
oc, clt = '1ALO', '0000170669'

In [ ]:
reload(utils)
test = utils.day_orders_pipe(
    data=day_orders.loc[oc, clt, :],
    inactive_duration=20.,
    indicator_status='brutrevenue',
    origin='WEB',
    indicator_perf='margin',
    inactive_roll_mode='ignore',
    roll_parms=dict(
        window=75,
        center=True,
        win_type='triang',
        min_periods=1,
    )
)
test

In [ ]:
%%time
# 30 seconds
day_orders = (
    day_orders.join(
        pd.concat([day_orders.groupby('indicators', axis=1).sum()], keys=['total'], axis=1).swaplevel(axis=1)
    ).sort_index(axis=1)
)
# day_orders

In [ ]:
reload(utils)

In [ ]:
%%time
# 1:30 mins
rolled = (
utils.compute_rolling_percentage(
    data=day_orders,
    window=100, 
    win_type='triang',
    indicator='brutrevenue',
    axis='WEB',
    roll_kwargs=dict(center=True),
    groupers=['orgacom', 'client'],
).rename('percentage')
)

In [ ]:
oc, clt = '1ALO', '0000020575'
oc, clt = '1ALO', '0000020531'
oc, clt = '1ALO', '0000024795'
oc, clt = '1ALO', '0000170669'
# oc, clt = list(rolled.loc[rolled != 0]
#                      .index.to_frame()
#                      .loc[lambda x: x.client.str[0] != 'P', ['orgacom', 'client']].drop_duplicates().sample(1).iloc[0])
fig, axs = plt.subplots(nrows=2, figsize=(15, 12), sharex=True)
day_orders.loc[(oc, clt), [('brutrevenue', 'total'), ('brutrevenue', 'WEB')]].plot(kind='line', ax=axs[0])
rolled.loc[(oc, clt)].plot(kind='line', ax=axs[1])
# ax.set_xlim(pd.to_datetime('2018-03-01'), pd.to_datetime('2018-05-31'))
axs[0].set_xticklabels([])
axs[1].set_ylim(0, 1.05)
print(oc, clt)
with pd.option_context('display.max_rows', None):
    display(orders.loc[(oc, clt), ['origin2', 'brutrevenue', ]])

In [ ]:
%%time
reload(utils)

inactive=utils.mask_successive_values(ds=day_orders[('brutrevenue', 'total')],
                                 groupers=['orgacom', 'client'], 
                                 count=20,
                                 value=0.,
                                )
inactive

In [ ]:
%%time
# 10 sec
full_stat=(
utils.compute_stat_from_percentage(
    rolled,
    mode='cut',
).astype('str').where(~inactive, 'inactive')
).rename('status')

In [ ]:
full_stat = full_stat.reset_index().sort_values(['orgacom', 'client', 'date'])

In [ ]:
reload(utils)
utils.compute_end_date(data=test.reset_index(), groupers=['orgacom', 'client'])

In [ ]:
%%time
# 40 sec
status_updates = utils.get_first_rupture_from_group(
                        df=full_stat.sample(frac=1),
                        groupers=['orgacom', 'client'],
                        order_keys=['date'],
                        targets=['status'],
                    ).set_index(['orgacom', 'client', 'date'])

In [ ]:
status_updates = status_updates.reset_index()
groupers = ['orgacom', 'client']
same_clt = (status_updates[groupers].shift(-1) == status_updates[groupers]).all(axis=1)
status_updates['end_date'] = status_updates['date'].shift(-1).where(same_clt, None)
status_updates

In [ ]:
%%time
# 2 min
roll_perf = day_orders.loc[:, idx[:, 'total']].groupby(['orgacom', 'client']).apply(
    lambda x: x.rolling(100, win_type='triang', min_periods=15, center=True).mean()
)
roll_perf

In [ ]:
%%time
# 2 min
roll_perf2 = day_orders.loc[:, idx[:, 'total']].where(~inactive, np.nan).groupby(['orgacom', 'client']).apply(
    lambda x: x.rolling(100, win_type='triang', min_periods=15, center=True).mean()
)
roll_perf2

In [ ]:
roll_perf.loc[oc, clt].plot(kind='line')
roll_perf2.loc[oc, clt].plot(kind='line')

In [ ]:
roll_perf2.columns = roll_perf2.columns.droplevel(1)
roll_perf2

In [ ]:
roll_perf2 = roll_perf2.reset_index()

In [17]:
for row in zbob.droplevel(1, axis=1).itertuples():
    print(row)

Pandas(Index=0, orgacom='1ALO', client='0000170669', date=Timestamp('2017-07-05 00:00:00'), status='no_web', end_date=Timestamp('2019-07-15 00:00:00'))
Pandas(Index=528, orgacom='1ALO', client='0000170669', date=Timestamp('2019-07-15 00:00:00'), status='adopt', end_date=Timestamp('2019-08-15 00:00:00'))
Pandas(Index=551, orgacom='1ALO', client='0000170669', date=Timestamp('2019-08-15 00:00:00'), status='exploit', end_date=Timestamp('2019-12-12 00:00:00'))
Pandas(Index=636, orgacom='1ALO', client='0000170669', date=Timestamp('2019-12-12 00:00:00'), status='exclusive', end_date=Timestamp('2020-02-11 00:00:00'))
Pandas(Index=679, orgacom='1ALO', client='0000170669', date=Timestamp('2020-02-11 00:00:00'), status='exploit', end_date=Timestamp('2020-03-16 00:00:00'))
Pandas(Index=703, orgacom='1ALO', client='0000170669', date=Timestamp('2020-03-16 00:00:00'), status='inactive', end_date=Timestamp('2020-05-22 00:00:00'))
Pandas(Index=752, orgacom='1ALO', client='0000170669', date=Timestamp('2

In [26]:
zbob

,orgacom,client,date,status,end_date
0,1ALO,0000170669,2017-07-05,no_web,2019-07-15
528,1ALO,0000170669,2019-07-15,adopt,2019-08-15
551,1ALO,0000170669,2019-08-15,exploit,2019-12-12
636,1ALO,0000170669,2019-12-12,exclusive,2020-02-11
679,1ALO,0000170669,2020-02-11,exploit,2020-03-16
703,1ALO,0000170669,2020-03-16,inactive,2020-05-22
752,1ALO,0000170669,2020-05-22,exploit,2020-06-29
778,1ALO,0000170669,2020-06-29,exclusive,NaT


In [25]:
reload(utils)
zbob = (partial(utils.bk_detail,
             data=day_orders.loc[oc, clt, :],
#              data=orders.reset_index().loc[
# #                  (orders.reset_index().date >= pd.to_datetime('2017-01-01')) &
# #                  (orders.reset_index().date <= pd.to_datetime('2019-12-31')) &
#                  (orders.reset_index().client == clt)
#              ],
             rolled=None, #rolled.rename('percentage').loc[(oc, clt)].reset_index(),
             client=clt,
             oc=oc,
             status_updates=None, #status_updates.loc[(status_updates.client == clt) & (status_updates.orgacom == oc)],
             roll_perf=None, #roll_perf2.loc[(roll_perf2.client == clt) & (roll_perf2.orgacom == oc)],
            )(None))

2020-10-22 10:04:40.277143: Computing totals
2020-10-22 10:04:40.283486: Done! Elapsed: 0:00:00.006353
2020-10-22 10:04:40.283585: Computing inactive periods
2020-10-22 10:04:40.288926: Done! Elapsed: 0:00:00.005349
2020-10-22 10:04:40.289102: Computing rolling indicators
2020-10-22 10:04:40.312749: Done! Elapsed: 0:00:00.023657
2020-10-22 10:04:40.312834: Computing percentage and statuses
2020-10-22 10:04:40.318753: Done! Elapsed: 0:00:00.005928


In [43]:
orders

margin  brutrevenue   weight  linecount  \
orgacom client     date                                                  
1SOU    0000010078 2017-07-17   11.01        20.82    2.350          2   
                   2017-11-06   12.23        27.83    1.450          1   
                   2017-11-20   13.92        26.43    2.850          3   
                   2018-10-01   58.89       117.10    2.200          4   
                   2019-11-18   80.70       169.10   10.400          6   
...                               ...          ...      ...        ...   
2BRE    SL009N     2020-06-23  265.71      1030.32  251.844          6   
                   2020-07-28   25.75       249.60   68.505          3   
                   2020-08-27   61.10       629.91  103.950          1   
                   2020-08-31   10.51        75.03   40.500          1   
        SLD008B    2018-09-14    3.82        70.05    4.400          1   

                              origin2 seg1 seg2 seg3 seg4 cat  ...  \
orgacom client     date                                        ...   
1SOU    0000010078 2017-07-17      TV   Z4   Z7   ZM   ZK  Z1  ...   
                   2017-11-06     WEB   Z4   Z7   ZM   ZK  Z1  ...   
                   2017-11-20     WEB   Z4   Z7   ZM   ZK  Z1  ...   
                   2018-10-01     WEB   Z4   Z7   ZM   ZK  Z1  ...   
                   2019-11-18     WEB   Z4   Z7   ZM   ZK  Z1  ...   
...                               ...  ...  ...  ...  ...  ..  ...   
2BRE    SL009N     2020-06-23      TV   Z4   Z9   ZO   ZM  Z3  ...   
                   2020-07-28      TV   Z4   Z9   ZO   ZM  Z3  ...   
                   2020-08-27      TV   Z4   Z9   ZO   ZM  Z3  ...   
                   2020-08-31      TV   Z4   Z9   ZO   ZM  Z3  ...   
        SLD008B    2018-09-14      TV   Z4   Z9   ZO   ZM  Z3  ...   

                               lineweight margin_clt_zscore  \
orgacom client     date                                       
1SOU    0000010078 2017-07-17    1.175000         -0.821021   
                   2017-11-06    1.450000         -0.775948   
                   2017-11-20    0.950000         -0.713512   
                   2018-10-01    0.550000          0.947901   
                   2019-11-18    1.733333          1.753669   
...                                   ...               ...   
2BRE    SL009N     2020-06-23   41.974000          1.092239   
                   2020-07-28   22.835000         -0.553079   
                   2020-08-27  103.950000         -0.310697   
                   2020-08-31   40.500000         -0.657574   
        SLD008B    2018-09-14    4.400000          0.000000   

                               brutrevenue_clt_zscore  PMVK_clt_zscore  \
orgacom client     date                                                  
1SOU    0000010078 2017-07-17               -0.840502        -0.651716   
                   2017-11-06               -0.714772         0.012193   
                   2017-11-20               -0.739882        -0.625110   
                   2018-10-01                0.886367         2.198822   
                   2019-11-18                1.819034        -0.176278   
...                                               ...              ...   
2BRE    SL009N     2020-06-23                2.809615        -0.078710   
                   2020-07-28               -0.207382        -0.160908   
                   2020-08-27                1.262280         0.282831   
                   2020-08-31               -0.881986        -0.489814   
        SLD008B    2018-09-14                0.000000         0.000000   

                               linecount_clt_zscore  marginperkg_clt_zscore  \
orgacom client     date                                                       
1SOU    0000010078 2017-07-17             -0.511372               -0.595015   
                   2017-11-06             -1.107972               -0.117863   
                   2017-11-20              0.085229       

In [53]:
reload(utils)
show(partial(
        utils.bk_detail,
        data=day_orders.loc[oc, clt, :],
        order_data=orders.loc[oc, clt, :],
        client=clt,
        oc=oc,
        bins=utils.def_bins,
        groupers=['orgacom', 'client'],
        inactive_duration=20,
        indicator_status='brutrevenue',
        origin='WEB',
        indicator_perf='margin',
        roll_parms=dict(
            window=150,
            center=True,
            win_type='triang',
            min_periods=1,
        ),
        inactive_roll_mode='stitch',
    ),
    notebook_url="http://devdm:8888")

2020-10-22 10:25:26.047932: Computing totals
2020-10-22 10:25:26.054381: Done! Elapsed: 0:00:00.006458
2020-10-22 10:25:26.054480: Computing inactive periods
2020-10-22 10:25:26.059878: Done! Elapsed: 0:00:00.005406
2020-10-22 10:25:26.060101: Computing rolling indicators
2020-10-22 10:25:26.083594: Done! Elapsed: 0:00:00.023502
2020-10-22 10:25:26.083662: Computing percentage and statuses
2020-10-22 10:25:26.089124: Done! Elapsed: 0:00:00.005471


In [ ]:
test = pd.Series(
    [1., 2., 2., np.nan, 1., 1.],
    index=range(6)
)

In [ ]:
test

In [ ]:
test.rolling(3, center=True, min_periods=1).mean()

In [ ]:
test = day_orders.loc[('1ALO', '0000020555'), [('brutrevenue', 'WEB'), ('brutrevenue', 'total')]]
test.columns = test.columns.droplevel(level=0)
test2 = test.rolling(30, min_periods=1, win_type='triang').sum()
(test2['WEB'] / test2['total']).fillna(0)

In [ ]:
day_orders.loc[('1ALO', '0000020555')]
rolled.loc[('1ALO', '0000020555')]

In [ ]:
rolled.loc[(rolled >= 1.)]

In [ ]:
rolled.loc[rolled != 0].index.get_level_values(1).unique()[:30]

In [ ]:
df = day_orders.loc[:, 'brutrevenue'].copy()
print(df.shape)
print(df.sum(axis=1).shape)
df['total'] = 3 #df.sum(axis=1).rename((indicator, 'total'))
# df = df[(indicator, axis), (indicator, 'total')]
df

In [ ]:
df[('brutrevenue', 'total')] = df.sum(axis=1).rename(('brutrevenue', 'total'))

In [ ]:
%%time
rolled2 = (
    toto.groupby(['orgacom', 'client'], observed=True)
        .transform(lambda x: x.rolling(5, win_type='triang').mean())
)
rolled2

In [ ]:
pd.concat(
    [day_orders.loc[idx['2BRE', 'SL008C', :], [('brutrevenue', 'WEB'), ('brutrevenue', 'total')]],
     day_orders.loc[idx['2BRE', 'SL009N', :], [('brutrevenue', 'WEB'), ('brutrevenue', 'total')]],
    ]
).groupby(['orgacom', 'client']).rolling(5, win_type='triang').mean()

In [ ]:
smol_data = day_orders.loc[[('2BRE', 'SL009N', slice(None)), ('2BRE', 'SL008C')]]

In [ ]:
smol_data = day_orders.loc[('2BRE', 'SL009N'), ('2BRE', 'SL008C')]
smol_data[[('brutrevenue', 'WEB'), ('brutrevenue', 'total')]].plot(kind='line')
smol_data.loc[smol_data[('brutrevenue', 'WEB')] != 0]

In [ ]:
smol_data[('brutrevenue', 'WEB')].rolling(5, win_type='triang').mean()

In [ ]:
rolled.loc[('2BRE', 'SL009N')].plot(kind='line')

In [ ]:
day_orders.loc[('2BRE', 'SL008R')]

In [ ]:
day_orders.loc[('2BRE')].index.get_level_values(0).unique()

In [ ]:
len(to_list[1])

In [ ]:
day_orders = day_orders.sort_index()

In [ ]:
%time
toto = day_orders.tail(50000).copy()
toto
rolled2 = (
    toto.loc[:, ('brutrevenue', 'WEB')]  # .loc[:, ['WEB', 'total']]
        .groupby(['orgacom', 'client'], observed=True)
        .rolling(5, win_type='triang')
        .mean()    
        .reset_index()
        .drop(['orgacom', 'client'], axis=1)
)

print(rolled2)

In [ ]:
rolled2.loc[rolled2[('brutrevenue', 'WEB')] > 0].index.get_level_values(1).unique()

In [ ]:
rolled2.loc[idx[:, '0000020511', :], ('brutrevenue', 'WEB')].plot(kind='line')

In [ ]:

# to_list = list(zip(*rolled2.level_2.to_list()))
# rolled2['orgacom'] = to_list[0]
# rolled2['client'] = to_list[1]
# rolled2['date'] = to_list[2]
# rolled2 = rolled2.drop('level_2', axis=1).set_index(['orgacom', 'client', 'date'])

In [ ]:
%%time
in_df = day_orders.tail(50000)
rolled = (
    in_df.loc[:, ('brutrevenue', 'WEB')]  # .loc[:, ['WEB', 'total']]
        .groupby(['orgacom', 'client'], observed=True)
        .rolling(5, win_type='triang')
        .mean()    
        .reset_index()
        .drop(['orgacom', 'client'], axis=1)
)

to_list = list(zip(*rolled.level_2.to_list()))
rolled['orgacom'] = to_list[0]
rolled['client'] = to_list[1]
rolled['date'] = to_list[2]
rolled = rolled.drop('level_2', axis=1).set_index(['orgacom', 'client', 'date'])
rolled 

In [ ]:
day_orders.loc[idx['2BRE', 'SL009N'], ('brutrevenue', 'WEB')].rolling(5,).mean()

In [ ]:
rolled.loc[idx['2BRE', 'SL009N']]

In [ ]:
(
day_orders.loc[:, ('brutrevenue', 'WEB')].to_frame()
.groupby(['orgacom', 'client']).rolling(5, win_type='triang').mean()
)

In [ ]:
(
day_orders.loc[idx['2BRE', ('SL009N', 'SL008R')], ('brutrevenue', 'WEB')].to_frame()
.groupby(['orgacom', 'client']).rolling(5, win_type='triang').mean()
)

In [ ]:
day_orders.loc[idx['2BRE', ('SL009N', 'SL008R')], ('brutrevenue', 'WEB')].to_frame().rolling(5, win_type='triang').mean()

In [ ]:
day_orders.loc[idx['2BRE', 'SL009N'], ('brutrevenue', 'WEB')].to_frame().rolling(5, win_type='triang').mean()

In [ ]:
rolled['percentage'] = rolled['WEB'] / rolled['total']
rolled

In [ ]:
rolled.loc[rolled.WEB > 0]

In [ ]:
orders = pd.read_pickle(persist_path / 'orders.pkl')

In [ ]:
rolled.loc[('2BRE', 'SL009N')]

In [ ]:
day_orders.loc[('2BRE', 'SL009N')].rolling(5, win_type='triang').mean()

In [ ]:
orders.loc[('2BRE', 'SL009N'), ['origin2', 'brutrevenue']]

In [ ]:
test1 = pd.DataFrame(
    {
        'rev': [*list([0.] * 6), 5., *list([0.] * 993)],
        'group': ['a'] * 1000,
        'date': pd.date_range(start=pd.to_datetime('2010-01-01'), periods=1000, freq='B')
    }
)
test2 = pd.DataFrame(
    {
        'rev': [0.] * 1000,
        'group': ['b'] * 1000,
        'date': pd.date_range(start=pd.to_datetime('2010-01-01'), periods=1000, freq='B')
    }
)
test = pd.concat([test1, test2], axis=0)
test = test.set_index(['group', 'date'])
test

In [ ]:
test.rolling(5, win_type='triang').mean().head(15)#.loc['b']

In [ ]:
test.groupby('group').rolling(5, win_type='triang').mean().head(15)#.loc['b']